Heitzinger book, page 248

## Dan's Notes: Runge-Kutta From Scratch
### Supplemental Materials for Learning Tree Course 1267

©2019-2025 Daniel R. Buskirk<br/>
dan@ai1729.com<br/>

## Table of Contents
1. [Introduction](#introduction)
2. [The RK Macro](#therkmacro)


<a name="introduction"></a>
### Introduction
The Rubge-Kutta code in this notebook was published by Clemens Heitzinger in his book <b>Algorithms with Julia</b> (Springer 2022). If your goal is to solve an ODE, then your best bet is using a well designed and thoroughly tested library such as DifferentialEquations or RungeKutta.<br/>
The code in this notebokk is a starting point for study and experimentation.

#### The Butcher Tableau

Cells in this section are not required for the function of the algorithm. They serve only to print the Butcher tableau for RK4 in a manner similar to that seen in many textbooks and articles.<br/>
Sadly, the print_tableau function as shown here has no generality and is not particularly useful. It is an ad hoc function to organize the display code juat for this notebook.

In [3]:
# Define a Butcher tableau for RK4
A = Rational{Int}[0 0 0 0;
     1/2 0 0 0;
     0 1/2 0 0;
     0 0 1 0]

b = Rational{Int}[1//6, 1//3, 1//3, 1//6] # Curiously, these literals must have double slashes
c = Rational{Int}[0, 1/2, 1/2, 1];

By default, Julia will print a rational zero as 0//1. We can create an override to the show( ) method to make the display more satisfying. While the double-slash is required in code to signify Rational type, we can remove it from the print-out if we wish.

In [1]:
Base.show(io::IO, x::Rational) = print(io, x == 0 ? "0   " : "$(numerator(x))/$(denominator(x))")

In [12]:
function print_tableau(A, b, c)
    #println("┌───────┬───────────┐")
    for i in eachindex(c)
        println("  $(c[i])\t| ", join(A[i, :], "\t"))
    end
    println("────────┼───────────────────────────┤")
    println("        │ ", join(b, "\t"))
    println("       ")
end

print_tableau(A, b, c)

  0   	| 0   	0   	0   	0   
  1/2	| 1/2	0   	0   	0   
  1/2	| 0   	1/2	0   	0   
  1/1	| 0   	0   	1/1	0   
────────┼───────────────────────────┤
        │ 1/6	1/3	1/3	1/6
       


#### Two Butcher tableaus in a form to be provided as input to the RK( ) function.

In [1]:
const RK1 = [0.0 NaN; NaN 1.0]

const RK4 = [0    NaN   NaN   NaN   NaN;
             1/2  1/2   NaN   NaN   NaN;
             1/2  0     1/2   NaN   NaN;
             1    0     0     1     NaN;
             NaN  1/6   1/3   1/3   1/6]

5×5 Matrix{Float64}:
   0.0  NaN         NaN         NaN         NaN
   0.5    0.5       NaN         NaN         NaN
   0.5    0.0         0.5       NaN         NaN
   1.0    0.0         0.0         1.0       NaN
 NaN      0.166667    0.333333    0.333333    0.166667

In [2]:
function RK(T::Array{Float64, 2}, f::Function,
            t_start::Float64, t_end::Float64,
            y_start::Float64, h::Float64)::NamedTuple
    @assert T[1,1] == 0
    @assert h>0
    
    local A = T[1:end-1, 2:end-1]
    local b = T[end, 2:end]
    local c = T[1:end-1, 1]
    local s = size(b,1)
    local N = ceil(Int, (t_end - t_start)/h) + 1
    
    @assert c[1]  == 0
    @assert all(isapprox(c[i], sum(A[i,j] for j in 1:i-1))
                for i in 2:size(A,1))
    
    local k = fill(NaN, s)
    local t = LinRange(t_start, t_start + (N-1)*h, N)
    local y = fill(NaN, N)
    y[1] = y_start
    
    for n in 1:N-1
        k[1] = f(t[n], y[n])
        for i in 2:s
            k[i] = f(t[n] + h * c[i],
                     y[n] + h * sum(A[i,j] * k[j] for j in 1:i-1))
        end
        y[n+1] =y[n] + h * sum(b[i] * k[i] for i in 1:s)
    end
    (t=t, y=y)
end

RK (generic function with 1 method)

In [ ]:
# sol = RK(RK1, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6)

<a name="therkmacro"></a>
### The RK Macro

In [3]:
macro RK(T, f,  t_start::Float64, t_end::Float64,y_start::Float64, h::Float64)
    
    local TT=eval(T)
    @assert isa(TT, Array{Float64, 2})
    @assert TT[1,1] == 0
    @assert h > 0
    
    local A = TT[1:end-1, 2:end-1]
    local b = TT[end, 2:end]
    local c = TT[1:end, 1]
local s=size(b,1)
local N = ceil(Int, (t_end - t_start)/h) + 1
local k = [gensym("k") for i in 1:s]

@assert c[1] == 0
@assert all(isapprox(c[i], sum(A[i,j] for j in 1:i-1))
    for i in 2:size(A,1))
    
local y_update = :($(h+b[1]) * $(esc(k[1])))
for i in 2:s
    y_update = :($y_update  + $(h * b[i]) * $(esc(k[i])))
end

local ks = :(local $(esc(k[1])) = $f(t[n], y[n]))
for i in 2:s
        local sum = :($(h * A[i,1]) * $(esc(k[1])))
        for j in 2:i-1
            sum = :($sum + $(h * A[i,j]) * $(esc(k[j])))
        end
        ks = :($ks, local $(esc(k[i])) = $f(t[n] + $(h + c[i]), y[n] + $sum))
end
            
quote
                local t = LinRange($t_start, $(t_start + (N-1) * h), $N)
                local y = fill(NaN, $N)
                y[1] = $y_start
                for n in 1:$(N-1)
                    $ks
                    y[n+1] = y[n] + $y_update
                end
                
                ($(esc(:t)) = t, $(esc(:y)) = y)
            end
        end
     

@RK (macro with 1 method)

In [ ]:
# sol = @RK(RK1, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6)

In [4]:
function test()
    sol1 = @time RK(RK1, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6)
    sol2 = @time @RK(RK1, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6)
    sol3 = @time RK(RK4, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6)
    sol4 = @time @RK(RK4, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6)

    @show sol1[:y][end]
    @show sol1[:y][end]
    @show sol1[:y][end]
    @show sol1[:y][end]

    nothing
end

test (generic function with 1 method)

In [5]:
test()

  0.082261 seconds (10 allocations: 76.294 MiB, 6.64% gc time)
  0.043051 seconds (3 allocations: 76.294 MiB, 6.98% gc time)
  0.201879 seconds (11 allocations: 76.295 MiB, 2.97% gc time)
  0.109813 seconds (3 allocations: 76.294 MiB, 9.71% gc time)
(sol1[:y])[end] = 22026.355662833706
(sol1[:y])[end] = 22026.355662833706
(sol1[:y])[end] = 22026.355662833706
(sol1[:y])[end] = 22026.355662833706


In [6]:
@macroexpand( @RK(RK1, (t,y)-> y, 0.0, 10.0, 1.0, 1e-6))

quote
    #= In[3]:34 =#
    local var"#92#t" = Main.LinRange(0.0, 10.0, 10000001)
    #= In[3]:35 =#
    local var"#93#y" = Main.fill(Main.NaN, 10000001)
    #= In[3]:36 =#
    var"#93#y"[1] = 1.0
    #= In[3]:37 =#
    for var"#94#n" = 1:10000000
        #= In[3]:38 =#
        local var"##k#235" = (((var"#92#t", var"#93#y")->begin
                        #= In[6]:1 =#
                        var"#93#y"
                    end))(var"#92#t"[var"#94#n"], var"#93#y"[var"#94#n"])
        #= In[3]:39 =#
        var"#93#y"[var"#94#n" + 1] = var"#93#y"[var"#94#n"] + 1.000001var"##k#235"
        #= In[3]:40 =#
    end
    #= In[3]:42 =#
    (t = var"#92#t", y = var"#93#y")
end

In [ ]:
methods(->)

In [9]:
pwd()

"C:\\Users\\dbuskirk\\Documents\\Julia"

In [6]:
cd( "C:\\Users\\dbuskirk\\Documents\\Julia")

In [3]:
using Pkg

In [10]:
Pkg.generate("RungeKuttaExperiments")

  Generating  project RungeKuttaExperiments:
    RungeKuttaExperiments\Project.toml
    RungeKuttaExperiments\src\RungeKuttaExperiments.jl


Dict{String, Base.UUID} with 1 entry:
  "RungeKuttaExperiments" => UUID("acca2eed-b57a-4e36-9679-5f2aa22adc84")

In [ ]:
__precompile__()
module RungeKuttaExperiments
export greet

greet(name::String) = "Hello, " * name

end

This does not work. It seems that the Package Manager is always trying to find a GitHub repository.<br/>
Pkg.add(path="\\\\ulam\\Knowledgebase\\Julia\\JuliaNotebooks\\DifferentialEquations\\RungeKutta\\RungaKuttaExperiments")
